# Models 1

## imports

In [9]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import SGDRegressor

from sklearn.model_selection import KFold, RandomizedSearchCV
import lightgbm as lgbm
from scipy import stats


## load data

In [33]:
dfcombo = pd.read_csv('combo.csv')
dfcombo.describe().T

,count,mean,std,min,25%,50%,75%,max
zipcode,217324.0,48161.549309,27812.434636,1001.000000,25315.000000,47171.000000,71241.000000,9.990100e+04
year,217324.0,2016.619789,2.857517,2012.000000,2014.000000,2017.000000,2019.000000,2.021000e+03
population,217324.0,14192.464555,15809.836341,25.000000,2512.000000,7483.000000,21630.000000,1.303520e+05
median_household_income,217324.0,60253.622380,24950.906483,2499.000000,43789.750000,54638.500000,70286.000000,2.500010e+05
median_age,217324.0,41.236737,6.945752,15.900000,36.700000,40.900000,45.200000,8.430000e+01
median_rent,217324.0,923.939557,374.824226,99.000000,671.000000,821.000000,1074.000000,3.501000e+03
mean_travel_time_to_work,217324.0,6207.458339,7099.169141,0.000000,1035.000000,3109.000000,9409.000000,6.095600e+04
median_value,217324.0,212311.416930,179707.665540,9999.000000,108500.000000,158200.000000,247500.000000,2.000001e+06
ave_num_rooms,217324.0,5.731863,0.833573,1.300000,5.300000,5.700000,6.200000,1.000000e+01
gini,217324.0,0.424868,0.055501,0.128000,0.388275,0.420900,0.457300,7.865000e-01


Per the previous notebook, 2019, 2020, and 2021 seem to be abormal. These correspond to COVID years (where 2019 represents price changes between Dec 2019 and Dec 2020). I will add one-hot encoding for those years. (to be used in linear regression and SGD)

Also, I will make "year" a categorical column (to be used in random forest and lightGBM) 

In [36]:
years_to_encode = [2019, 2020, 2021]
for year in years_to_encode:
    dfcombo[f'year_{year}'] = (dfcombo['year'] == year).astype(int)
# dfcombo.sample(10)

,zipcode,year,population,median_household_income,median_age,median_rent,mean_travel_time_to_work,median_value,ave_num_rooms,gini,...,pct_next_4yr,pct_next_5yr,pct_next_6yr,pct_next_7yr,pct_next_8yr,pct_next_9yr,pct_next_10yr,year_2019,year_2020,year_2021
1986,16201,2012,18426,44763.0,45.8,575.0,7931.0,104600.0,5.8,0.4145,...,0.126640,0.167731,0.131445,0.239899,0.414960,0.571076,0.687931,0,0,0
41886,4551,2014,894,55750.0,47.5,1091.0,356.0,225800.0,5.4,0.4074,...,0.267705,0.344546,0.558175,0.974498,1.162124,NaN,NaN,0,0,0
124260,86436,2017,1308,26791.0,62.8,761.0,322.0,77400.0,4.0,0.4069,...,0.647710,0.808998,NaN,NaN,NaN,NaN,NaN,0,0,0
60241,95832,2014,11908,42228.0,27.2,1120.0,4042.0,160700.0,5.5,0.4086,...,0.554611,0.654355,0.836344,1.195734,1.209971,NaN,NaN,0,0,0
54644,63461,2014,6022,50106.0,40.6,553.0,2829.0,114900.0,5.7,0.4421,...,0.192430,0.226491,0.341607,0.542104,0.623986,NaN,NaN,0,0,0
92640,18337,2016,14303,75313.0,44.0,1114.0,6525.0,227500.0,6.0,0.3958,...,0.320988,0.593254,0.714735,NaN,NaN,NaN,NaN,0,0,0
162093,52746,2019,1067,84688.0,47.9,881.0,486.0,176400.0,6.4,0.4106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
192273,88260,2020,14836,62815.0,32.0,810.0,5580.0,128000.0,5.3,0.4290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
98789,57106,2016,44575,58837.0,33.6,791.0,24590.0,163400.0,5.5,0.3976,...,0.264947,0.445299,0.598684,NaN,NaN,NaN,NaN,0,0,0
85180,25304,2016,8321,53125.0,42.9,698.0,3729.0,183800.0,5.9,0.5246,...,0.025238,0.118482,0.178054,NaN,NaN,NaN,NaN,0,0,0


In [37]:
dfcombo['year'] = dfcombo['year'].astype('category')

* featuresA to be use in linear regression and SGD (one-hot encoding of years)
* featuresB to be used in random forest and lightGBM (year as categorixal variable)

In [40]:
featuresX = dfcombo.columns.tolist()
del featuresX[31:]
del featuresX[0:2]
featuresA = featuresX + ['year_2019', 'year_2020', 'year_2021']
featuresB = featuresX + ['year']
ycol='pct_next_1yr'
y = [ycol]
featuresA

['population',
 'median_household_income',
 'median_age',
 'median_rent',
 'mean_travel_time_to_work',
 'median_value',
 'ave_num_rooms',
 'gini',
 'cost_of_living_perc',
 'median_RE_tax',
 'labor_force_perc',
 'unemployed_perc',
 'bach_degr_perc',
 'masters_degr_perc',
 'peops_per_household',
 'owner_occ_perc',
 'new_units_perc',
 'families_wU18_perc',
 'poverty_perc',
 'non_families_perc',
 'vacant_perc',
 'perc_moved_fr_same_county',
 'perc_moved_fr_other_county',
 'perc_moved_fr_other_state',
 'perc_moved_fr_abroad',
 'single_fam_perc',
 'public_trans_perc',
 'foreign_born_perc',
 'male_perc',
 'year_2019',
 'year_2020',
 'year_2021']

## linear regression

In [41]:
df_lr = dfcombo.dropna(subset=y+featuresA).copy()
scaler=StandardScaler()
df_lr[featuresA] = scaler.fit_transform(df_lr[featuresA])
train, temp = train_test_split(df_lr, test_size=0.2, random_state=42)
test, val = train_test_split(temp, test_size=0.5, random_state=42)

X_train, y_train = train[featuresA], train[ycol]
X_test, y_test = test[featuresA], test[ycol]
X_val, y_val = val[featuresA], val[ycol]

# 4. Perform linear regression
model = LinearRegression()
model.fit(X_train, y_train)

# Coefficients and intercept
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# Predictions on the validation set
y_val_pred = model.predict(X_val)

# Performance metrics on validation set
print("Validation Mean Squared Error:", mean_squared_error(y_val, y_val_pred))
print("Validation R^2 Score:", r2_score(y_val, y_val_pred))

Coefficients: [ 0.02575564  0.00295885  0.0045707   0.00095735 -0.02483927 -0.00223225
 -0.00082236 -0.00100647  0.00876684 -0.01091657  0.00042327  0.00141879
  0.00428933 -0.00283555  0.00111581 -0.00177144  0.00126464  0.0021219
  0.00123232  0.00062886 -0.00199209  0.00412314  0.00116142  0.00137308
 -0.00102011  0.00331685 -0.00249056  0.00776216  0.00010761  0.01813233
  0.02755689  0.01104623]
Intercept: 0.07257931847840812
Validation Mean Squared Error: 0.002619342181344577
Validation R^2 Score: 0.29033849842290627


When I added the year dummies, the R2 went from .09 to .29

## SGDRegressor

In [42]:
sgd_regressor = SGDRegressor(random_state=42)   # max_iter=1000, tol=1e-3, 
sgd_regressor.fit(X_train, y_train)

# Predictions on the validation set
y_val_pred = sgd_regressor.predict(X_val)

# Performance metrics on validation set
print("Validation Mean Squared Error:", mean_squared_error(y_val, y_val_pred))
print("Validation R^2 Score:", r2_score(y_val, y_val_pred))

Validation Mean Squared Error: 0.00262936599866485
Validation R^2 Score: 0.2876227336397853


When I added the year dummies, the R2 went from .09 to .29

## redo training and test data for random forest and lightGMB because year is now handled as categorical

In [44]:
df_lr = dfcombo.dropna(subset=y+featuresB).copy()
scaler=StandardScaler()
df_lr[featuresB] = scaler.fit_transform(df_lr[featuresB])
train, temp = train_test_split(df_lr, test_size=0.2, random_state=42)
test, val = train_test_split(temp, test_size=0.5, random_state=42)

X_train, y_train = train[featuresB], train[ycol]
X_test, y_test = test[featuresB], test[ycol]
X_val, y_val = val[featuresB], val[ycol]

## random forest regressor <span style="color:red">(slow)</span>

In [45]:
rf_regressor = RandomForestRegressor(random_state=42) 
                            # n_estimators=100, <- default, slow, R2=.238. with year 0.51
                            # n_estimators=10, <-         faster, R2=.154
rf_regressor.fit(X_train, y_train)

# Predictions on the validation set
y_val_pred = rf_regressor.predict(X_val)
print("Validation Mean Squared Error:", mean_squared_error(y_val, y_val_pred))
print("Validation R^2 Score:", r2_score(y_val, y_val_pred))

Validation Mean Squared Error: 0.0017972674753294884
Validation R^2 Score: 0.5130641791049713


## lightGBM with KFold and RandomizedSearchCV

In [48]:
X=dfcombo[featuresB].copy()
y=dfcombo[[ycol]].copy()
y.describe().T

,count,mean,std,min,25%,50%,75%,max
pct_next_1yr,217324.0,0.072615,0.061442,-0.395326,0.033453,0.064001,0.10372,0.75288


In [52]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217324 entries, 0 to 217323
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   population                  217324 non-null  int64   
 1   median_household_income     217324 non-null  float64 
 2   median_age                  217324 non-null  float64 
 3   median_rent                 217324 non-null  float64 
 4   mean_travel_time_to_work    217324 non-null  float64 
 5   median_value                217324 non-null  float64 
 6   ave_num_rooms               217324 non-null  float64 
 7   gini                        217324 non-null  float64 
 8   cost_of_living_perc         217324 non-null  float64 
 9   median_RE_tax               217324 non-null  float64 
 10  labor_force_perc            217324 non-null  float64 
 11  unemployed_perc             217324 non-null  float64 
 12  bach_degr_perc              217324 non-null  float64 
 13 

In [53]:
X_scaled = scaler.fit_transform(X)

# needed?
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


param_dist_old = {'n_estimators': [50, 75, 100, 150, 200, 300],
              'learning_rate': stats.uniform(0.01, 0.2),
              'objective': ["regression","poisson", "tweedie"#"gamma"
              #, "tweedie"
              ],
              'max_depth': [None] + [x for x in range(1,12)],
              'num_leaves': [x for x in range(10, 40, 3)],
              'min_data_in_leaf': [20,100,250],
              'reg_alpha': np.linspace(0,2, num=4),
              'reg_lambda': np.linspace(0,2, num=4),
              'subsample': [0.8, 1],
              'colsample_bytree': [0.8,1],
              'boosting_type': ["gbdt","dart"],
              'random_state':[2102021],
              #'n_jobs':[5],
}

param_dist = {# 'n_estimators': [50],
              # 'learning_rate': [.15], # stats.uniform(0.05, 0.2),
              # 'objective': ["regression",    # "poisson", "tweedie"#"gamma"
              #, "tweedie"
              # ],
              # 'max_depth': [6,12],# [None] + [x for x in range(1,12)],
              'num_leaves': [31,62,127], # [x for x in range(10, 40, 3)],
              'min_data_in_leaf': [20,250],
              'reg_alpha': [.1,.5], #np.linspace(0,2, num=4),
              # 'reg_lambda': [0,2], # np.linspace(0,2, num=4),
              'lambda_l1': [0,1,1.5],
              'lambda_l2': [0,1],
              # 'subsample': [0.8, 1],
              # 'colsample_bytree': [0.8,1],
              # 'boosting_type': ["gbdt"],
              'random_state':[42],
              #'n_jobs':[5],
}


myfolds = KFold(n_splits=5)


lgbm_mod = lgbm.LGBMRegressor()
# rand_cv_hist = RandomizedSearchCV(lgbm_mod, param_dist, cv=myfolds, n_jobs=1, n_iter=1000, scoring='neg_mean_absolute_error')
rand_cv_hist = RandomizedSearchCV(lgbm_mod, param_dist, cv=myfolds, n_jobs=-1, n_iter=50, scoring='r2', random_state=42, verbose=1)

In [54]:
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

rand_cv_hist.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when 

/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when 

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] min_data_in_leaf is set=250, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=250
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] min_data_in_leaf is set=250, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=250
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032784 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7225
[LightGBM] [Info] Number of data points in the train set: 139087, number of used features: 30


/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was 

...

/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when 

[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044410 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7225
[LightGBM] [Info] Number of data points in the train set: 139087, number of used features: 30
[LightGBM] [Info] Start training from score 0.072531
[LightGBM] [War

/Users/paulschickler/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001164 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7226
[LightGBM] [Info] Number of data points in the train set: 173859, number of used features: 30
[Lig

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=LGBMRegressor(), n_iter=50, n_jobs=-1,
                   param_distributions={'lambda_l1': [0, 1, 1.5],
                                        'lambda_l2': [0, 1],
                                        'min_data_in_leaf': [20, 250],
                                        'num_leaves': [31, 62, 127],
                                        'random_state': [42],
                                        'reg_alpha': [0.1, 0.5]},
                   random_state=42, scoring='r2', verbose=1)

In [55]:
# save CV result to csv 
# pd.DataFrame(rand_cv_hist.cv_results_).to_csv("LightGBM-Hyper-CV.csv")

best_model = rand_cv_hist.best_estimator_
# Figure out the model that you want to interpret
# fit the best model for that comment code
# interpret as below

# lgbm = bestmodel.fit(X,Y)

y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")

# variable importance from lgbm
# lgbm.plot_importance()
# plot_importance(best_model)
# plt.show()


[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
R2 Score: 0.503668893989339


R^2 went from .233 without year, to .504 with year